In [ ]:
# Import packages
from jupyter_dash import JupyterDash
from dash import Dash, html, dcc, callback, Output, Input
import pandas as pd
import plotly.express as px

In [1]:
from CCDPApy import CellLine    # used to aggregate all cell lines and experiments data
from CCDPApy import bioprocess_pipeline # used for data process of one experiment

species = ['Glucose','Lactate','Glutamine','Asparagine','Aspartate']

data_file_1 = ['CL1_1.xlsx', 'CL1_2.xlsx', 'CL1_3.xlsx']
key_cl1 = {'use_feed_conc': True,
           'use_conc_after_feed': False,
           'spc_list': species,
           'polyreg': True, 'polyorder_file': 'polynomial_order_1.xlsx',
           'rollreg': True, 'rollreg_order': 3, 'rollreg_window': 6,
           }

cell_line = CellLine()

for input in data_file_1:
    cl_1 = bioprocess_pipeline(input_file_name=input, **key_cl1)
    cell_line.add_bio_process(bio_process=cl_1)

data_file_2 = ['CL2_1.xlsx', 'CL2_2.xlsx', 'CL2_3.xlsx']
key_cl2 = {'use_feed_conc': False,
           'use_conc_after_feed': True,
           'spc_list': species,
           'polyreg': True, 'polyorder_file': 'polynomial_order_2.xlsx',
           'rollreg': True, 'rollreg_order': 3, 'rollreg_window': 6,
           }
for input in data_file_2:
    cl_2 = bioprocess_pipeline(input_file_name=input, **key_cl2)
    cell_line.add_bio_process(bio_process=cl_2)

CL1_1.xlsx imported.
In-Process Done.
Two-Point Calculations. Done.
Polynomial Regression. Done
Rolling Regression. Done.
CL1_2.xlsx imported.
In-Process Done.
Two-Point Calculations. Done.
Polynomial Regression. Done
Rolling Regression. Done.
CL1_3.xlsx imported.
In-Process Done.
Two-Point Calculations. Done.
Polynomial Regression. Done
Rolling Regression. Done.
CL2_1.xlsx imported.
In-Process Done.
Two-Point Calculations. Done.
Polynomial Regression. Done
Rolling Regression. Done.
CL2_2.xlsx imported.
In-Process Done.
Two-Point Calculations. Done.
Polynomial Regression. Done
Rolling Regression. Done.
CL2_3.xlsx imported.
In-Process Done.
Two-Point Calculations. Done.
Polynomial Regression. Done
Rolling Regression. Done.


In [2]:
cell_line.interactive_plot()

Dash is running on http://127.0.0.1:8050/



     Cell Line Experiment ID  RUN TIME (HOURS)      CONC.  Species
0   Sample CL1  Sample CL1_1               0.0  27.502389  GLUCOSE
1   Sample CL1  Sample CL1_1               0.0  27.502389  GLUCOSE
10  Sample CL1  Sample CL1_2               0.0  29.362180  GLUCOSE
11  Sample CL1  Sample CL1_2               0.0  29.362180  GLUCOSE
20  Sample CL1  Sample CL1_3               0.0  30.292953  GLUCOSE
     Cell Line Experiment ID  RUN TIME (HOURS)      CONC.  Species
0   Sample CL1  Sample CL1_1               0.0  27.502389  GLUCOSE
1   Sample CL1  Sample CL1_1               0.0  27.502389  GLUCOSE
10  Sample CL1  Sample CL1_2               0.0  29.362180  GLUCOSE
11  Sample CL1  Sample CL1_2               0.0  29.362180  GLUCOSE
20  Sample CL1  Sample CL1_3               0.0  30.292953  GLUCOSE
     Cell Line Experiment ID  RUN TIME (HOURS)      CONC.  Species
0   Sample CL1  Sample CL1_1               0.0  27.502389  GLUCOSE
1   Sample CL1  Sample CL1_1               0.0  27.502389  GLU

In [ ]:
cell_lines = cell_line.get_cell_line_list()
cell_lines

In [ ]:
exp_options = {}
for cl in cell_line.get_cell_line_list():
    exp_lst = list(cell_line.get_cell_line(cl).keys())
    exp_options[cl] = [exp for exp in exp_lst]
exp_options

In [ ]:
profiles = ['Concentraion', 'Cumulative Cons./Prod.', 'Sp. Rate']
methods = [{'label': 'Two-Point Calculation', 'value': 'Two-Pt. Calc.'}, 
           {'label': 'Polynomial Regression', 'value': 'Poly. Reg.'}, 
           {'label': 'Rolling Regression', 'value': 'Roll. Reg.'}]
plot_style_options = [{'label': 'Cell Line', 'value': 'Cell Line'},
                      {'label': 'Experiment ID', 'value': 'Experiment ID'},]
plot_style_options2 = [{'label': 'Cell Line', 'value': 'Cell Line'},
                       {'label': 'Experiment ID', 'value': 'Experiment ID'},
                       {'label': 'Method', 'value': 'Method'}]

In [ ]:
conc_df = cell_line.get_conc_df()
cumu_df = cell_line.get_cumu_df()
rate_df = cell_line.get_sp_rate_df()

# Initialize the app
app = JupyterDash(__name__)

# App layout
app.layout = html.Div([
    html.Div(
        children=[
            html.Div(
                children=[
                    html.Div("Cell Line: ", style={"margin-right": "10px", "font-weight": "bold"}),
                    dcc.Dropdown(
                        options=list(exp_options.keys()),
                        value=[cell_lines[0]],
                        placeholder='Select Cell Lines',
                        id='cell-line-dropdown',
                        multi=True, #style={"width": "400px"},
                    ),
                ],
                style={'width': '50%', 'display': 'inline-block', 'margin-right': '20px'},
            ),
            html.Div(
                children=[
                    html.Div("Experiment ID: ", style={"margin-right": "10px", "font-weight": "bold"}),
                    dcc.Dropdown(
                        options=exp_options[cell_lines[0]],
                        value=exp_options[cell_lines[0]],
                        placeholder='Select Experiment ID',
                        id='experiment-dropdown', 
                        multi=True, #style={"width": "600px"},
                    ),
                ],
                style={'width': '50%', 'display': 'inline-block'},
            ),
        ],
        style={'display': 'flex', 'align-items': 'center'},
    ),
    html.Hr(),
    html.Div(
        children=[
            html.Div(
                children=[
                    html.Div("Species: ", style={"margin-right": "10px", "font-weight": "bold"}),
                    dcc.Dropdown(
                        options=species,
                        value=[species[0]],
                        placeholder='Select Species',
                        id='species-dropdown',
                        multi=True, #style={"width": "500px"},
                    ),
                ],
                style={'width': '50%', 'display': 'inline-block', 'margin-right': '20px'},
            ),
            html.Div(
                children=[
                    html.Div("SP. Rate Method: ", style={"margin-right": "10px", "font-weight": "bold"}),
                    dcc.Dropdown(
                        options=methods,
                        value=['Two-Pt. Calc.'],
                        placeholder='Select Method',
                        id='method-dropdown',
                        multi=True, #style={"width": "500px"},
                    ),
                ],
                style={'width': '50%', 'display': 'inline-block'},
            ),  
        ],
        style={'display': 'flex', 'align-items': 'center'},
    ),
    html.Hr(),
    html.Div(
        children=[
            html.Div(
                children=[
                    html.Div("Color: ", style={"margin-right": "10px", "font-weight": "bold"}),
                    dcc.RadioItems(id="color-mode1", value='Cell Line', options=plot_style_options, inline=True,),
                ],
                style={'display': 'inline-block', 'margin-right': '20px'},
            ),
            html.Div(
                children=[
                    html.Div("Line Style: ", style={"margin-right": "10px", "font-weight": "bold"}),
                    dcc.RadioItems(id="line-style-mode1", value='Experiment ID', options=plot_style_options, inline=True,),
                ],
                style={'display': 'inline-block', 'margin-right': '20px'},
            ),
        ],
        style={'display': 'flex', 'align-items': 'center'},
    ),

    html.Hr(),
    html.Div(
        children=[
            html.Div(
                children=[
                    html.Div("SP. Rate Color: ", style={"margin-right": "10px", "font-weight": "bold"}),
                    dcc.RadioItems(id="color-mode2", value='Cell Line', options=plot_style_options2, inline=True,),
                ],
                style={'display': 'inline-block', 'margin-right': '20px'},
            ),
            html.Div(
                children=[
                    html.Div("SP. Rate Line Style: ", style={"margin-right": "10px", "font-weight": "bold"}),
                    dcc.RadioItems(id="line-style-mode2", value='Experiment ID', options=plot_style_options2, inline=True,),
                ],
                style={'display': 'inline-block', 'margin-right': '20px'},
            ),
            html.Div(
                children=[
                    html.Div("SP. Rate Symbol: ", style={"margin-right": "20px", "font-weight": "bold"}),
                    dcc.RadioItems(id="symbol-mode2", value='Method', options=plot_style_options2, inline=True,),
                ],
                style={'display': 'inline-block',},
            ),
        ],
        style={'display': 'flex', 'align-items': 'center'},
    ),    
    html.Hr(),

    html.Div(children='Legend:', style={"font-weight": "bold"}),
    dcc.RadioItems(id='legend-radio', options=['on', "off"], value='on', inline=True,),
    dcc.RadioItems(id='xanchor', value=0, inline=True, options=[{'label': 'left', 'value': 0}, {'label': 'right', 'value': 1}]),
    dcc.RadioItems(id='yanchor', value=1, inline=True, options=[{'label': 'top', 'value': 1}, {'label': 'bottom', 'value': 0}],),
    
    html.Hr(),    
    html.Div(children=[
        dcc.Graph(figure={}, id='figure1')
    ], style={'width': '33%', 'display': 'inline-block'}),
    html.Div(children=[
        dcc.Graph(figure={}, id='figure2')
    ], style={'width': '33%', 'display': 'inline-block'}),
    html.Div(children=[
        dcc.Graph(figure={}, id='figure3')
    ], style={'width': '33%', 'display': 'inline-block'})
    ])

# Update expriment ID dropdown
@app.callback(
    Output(component_id='experiment-dropdown', component_property='options'),
    Input(component_id='cell-line-dropdown', component_property='value'),
)
def set_exp_options(cl_chosen):
    available_exp = []
    for cl_name in cl_chosen:
        available_exp += exp_options[cl_name]
    return [{'label': i, 'value': i} for i in available_exp]

@app.callback(
    Output(component_id='experiment-dropdown', component_property='value'),
    Input(component_id='experiment-dropdown', component_property='options'),
)
def set_exp_values(exp_chosen):
    return [options['value'] for options in exp_chosen]

@app.callback(
    Output(component_id='figure1', component_property='figure'),
    Input(component_id='cell-line-dropdown', component_property='value'),
    Input(component_id='experiment-dropdown', component_property='value'),
    Input(component_id='species-dropdown', component_property='value'),
    Input(component_id='color-mode1', component_property='value'),
    Input(component_id='line-style-mode1', component_property='value'),
    Input(component_id='legend-radio', component_property='value'),
    Input(component_id='xanchor', component_property='value'),
    Input(component_id='yanchor', component_property='value'),
)
def conc_graph(cl_chosen, exp_chosen, spc_chosen, color_chosen, line_style_chosen, legend, pos_x, pos_y):
    if spc_chosen==[] or cl_chosen==[] or exp_chosen==[]:
        return {}
    # Filter df by cell lines
    cl_filter = filter(conc_df, 'Cell Line', cl_chosen)
    exp_filter = filter(conc_df, 'Experiment ID', exp_chosen)
    spc_filter = filter(conc_df, 'Species', spc_chosen)
    mask = cl_filter & exp_filter & spc_filter

    df = conc_df[mask]
    fig = px.line(df, x='RUN TIME (HOURS)', y='CONC.', markers=True,
                  title="Concentration (mmol)",
                  line_dash=line_style_chosen, color=color_chosen,
                  facet_row='Species')
    
    fig.update_layout(height=500*len(spc_chosen), width=500)
    fig.update_yaxes(matches=None)
    if legend=="on":
        fig.update_layout(showlegend=True, legend_x=pos_x, legend_y=pos_y,
                          #font=dict(size=3,),
                          )
    else:
        fig.update_layout(showlegend=False)
    fig.update_yaxes(title=None, visible=True)
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    fig.for_each_annotation(lambda a: a.update(xshift=-390))
    return fig


@app.callback(
    Output(component_id='figure2', component_property='figure'),
    Input(component_id='cell-line-dropdown', component_property='value'),
    Input(component_id='experiment-dropdown', component_property='value'),
    Input(component_id='species-dropdown', component_property='value'),
    Input(component_id='color-mode1', component_property='value'),
    Input(component_id='line-style-mode1', component_property='value'),
    Input(component_id='legend-radio', component_property='value'),
    Input(component_id='xanchor', component_property='value'),
    Input(component_id='yanchor', component_property='value'),
)
def cumu_graph(cl_chosen, exp_chosen, spc_chosen, color_chosen, line_style_chosen, legend, pos_x, pos_y):
    if spc_chosen==[] or cl_chosen==[] or exp_chosen==[]:
        return {}
    # Filter df by cell lines
    cl_filter = filter(cumu_df, 'Cell Line', cl_chosen)
    exp_filter = filter(cumu_df, 'Experiment ID', exp_chosen)
    spc_filter = filter(cumu_df, 'Species', spc_chosen)
    mask = cl_filter & exp_filter & spc_filter

    df = cumu_df[mask]
    fig = px.scatter(df[df['Method']=='Two-Pt. Calc.'], x='RUN TIME (HOURS)', y='Cumulative Prod./Cons.',
                     title="Cumulative Cons./Prod. (mmol)",
                     color=color_chosen,
                     facet_row='Species')

    fig2 = px.line(df[df['Method']=='Poly. Reg.'], x='RUN TIME (HOURS)', y='Cumulative Prod./Cons.',
                  line_dash=line_style_chosen, color=color_chosen,
                  facet_row='Species')
    for fig_data in fig2.data:
        fig.add_trace(fig_data)
    
    fig.update_layout(height=500*len(spc_chosen), width=500)
    fig.update_yaxes(matches=None)
    if legend=="on":
        fig.update_layout(showlegend=True, legend_x=pos_x, legend_y=pos_y,
                          #font=dict(size=3,),
                          )
    else:
        fig.update_layout(showlegend=False)
    fig.update_yaxes(title=None, visible=True)
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    fig.for_each_annotation(lambda a: a.update(xshift=-380))
    return fig

@app.callback(
    Output(component_id='figure3', component_property='figure'),
    Input(component_id='cell-line-dropdown', component_property='value'),
    Input(component_id='experiment-dropdown', component_property='value'),
    Input(component_id='species-dropdown', component_property='value'),
    Input(component_id='method-dropdown', component_property='value'),
    Input(component_id='color-mode2', component_property='value'),
    Input(component_id='line-style-mode2', component_property='value'),
    Input(component_id='symbol-mode2', component_property='value'),
    Input(component_id='legend-radio', component_property='value'),
    Input(component_id='xanchor', component_property='value'),
    Input(component_id='yanchor', component_property='value'),
)
def sp_rate_graph(cl_chosen, exp_chosen, spc_chosen, method_chosen, color_chosen, line_style_chosen, symbol_chosen, legend, pos_x, pos_y):
    if spc_chosen==[] or cl_chosen==[] or exp_chosen==[] or method_chosen==[]:
        return {}
    # Filter df by cell lines
    cl_filter = filter(rate_df, 'Cell Line', cl_chosen)
    exp_filter = filter(rate_df, 'Experiment ID', exp_chosen)
    spc_filter = filter(rate_df, 'Species', spc_chosen)
    method_filter = filter(rate_df, 'Method', method_chosen)
    mask = cl_filter & exp_filter & spc_filter & method_filter

    df = rate_df[mask]
    fig = px.line(df, x='RUN TIME (HOURS)', y='Sp. rate', markers=True,
                  title='SP. Rate (mmol/10^9 cell/hr)',
                  line_dash=line_style_chosen, color=color_chosen, symbol=symbol_chosen,
                  facet_row='Species')
    
    fig.update_layout(height=500*len(spc_chosen), width=500)
    fig.update_yaxes(matches=None)
    if legend=="on":
        fig.update_layout(showlegend=True, legend_x=pos_x, legend_y=pos_y,
                          #font=dict(size=3,),
                          )
    else:
        fig.update_layout(showlegend=False)
    fig.update_yaxes(title=None, visible=True)
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    fig.for_each_annotation(lambda a: a.update(xshift=-380))
    
    return fig

def filter(df, col, values):
    if values == []:
        values = [None]
    mask = df[col] == values[0]
    for i in range(1, len(values)):
        mask = mask | (df[col]==values[i])
    return mask

# Run app and display result inline in the notebook
app.run_server(mode="inline")